In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline
import pandas as pd
import os

Populating the interactive namespace from numpy and matplotlib


In [2]:
import sys
sys.path.append("/home/muncok/DL/projects/sv_system/")

In [3]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [4]:
from honk_sv import train as train
from honk_sv import model as mod
from honk_sv import dataset as dset
from honk_sv import system as svs
from honk_sv import dataloader as dloader

# Dataframes
command: disjoint speaker split

In [5]:
dataframe_dir = '/home/muncok/DL/dataset/SV_sets/dataframes/'
data_dir = '/home/muncok/DL/dataset/SV_sets/reddots_r2015q4_v1/wav/'
data_df = pd.read_pickle('/home/muncok/DL/dataset/SV_sets/dataframes/Reddots_Dataframe.pkl')

### Dataset split

In [6]:
#TODO: correct split
all_spks = list(data_df.spk.unique())
all_sents = data_df.sent.unique()
uttrs_counts = data_df.spk.value_counts()

In [7]:
train_spks = list(uttrs_counts[uttrs_counts > 100].index)

In [8]:
test_spks = list(uttrs_counts[uttrs_counts <= 100].index)

# SI Train

In [11]:
import random 

si_train = data_df[data_df.spk.isin(train_spks)]
si_val = si_train.sample(frac=0.2)
si_train = si_train.drop(index=si_val.index)
si_test = data_df[data_df.spk.isin(test_spks)]

In [13]:
import csv
import random

tags = ['train', 'val', 'test']
# choose a si split
sets = {'train':si_train, 'val':si_val, 'test':si_test}
manifest_dir = "../manifests/reddots/fewshot"

for tag in tags:
    samples = []
    save_path = os.path.join(manifest_dir,'si_{}_{}_manifest.csv'.format("reddots", tag))
    with open(save_path, 'w') as f:
        for index, row in sets[tag].iterrows():
            file_path = os.path.join(data_dir, row.spk, row.file)
            label = all_spks.index(row.spk)
            sample = ','.join([file_path, str(label)])
            samples.append(sample)
        random.shuffle(samples)
        writer = csv.writer(f, delimiter='\n', quoting=csv.QUOTE_NONE)
        writer.writerow(samples)
        print("{} was written".format(save_path))

../manifests/reddots/fewshot/si_reddots_train_manifest.csv was written
../manifests/reddots/fewshot/si_reddots_val_manifest.csv was written
../manifests/reddots/fewshot/si_reddots_test_manifest.csv was written


In [14]:
model = "SimpleCNN"
dataset = "reddots"

global_config = dict(model=model, dataset=dataset,
                     no_cuda=False,  gpu_no=0,
                     n_epochs=100, batch_size=64,
                     lr=[0.01], schedule=[np.inf], dev_every=1, seed=0, use_nesterov=False,
                     cache_size=32768, momentum=0.9, weight_decay=0.00001,
                     num_workers=16, print_step=100,
                     splice_length=20
                     )

builder = train.ConfigBuilder(
                dset.SpeechDataset.default_config(dataset),
                global_config)
parser = builder.build_argparse()
si_config = builder.config_from_argparse(parser)
si_config['model_class'] = mod.SimpleCNN
train.set_seed(si_config)

In [15]:
si_config['n_labels'] = len(all_spks)
si_config['input_length'] = int(16000*1)
manifest_dir = "../manifests/reddots/fewshot/"
for tag in ['train', 'val', 'test']:
    si_config['{}_manifest'.format(tag)]=os.path.join(manifest_dir,'si_{}_{}_manifest.csv'.format("reddots", tag))

In [16]:
si_model = si_config['model_class']()

In [17]:
from torch.autograd import Variable
import torch
import torch.nn as nn

test_in = Variable(torch.zeros(1,1,101,40), volatile=True)
test_out = si_model(test_in)
si_model.feat_size = test_out.size(1)
si_model.output = nn.Linear(test_out.size(1), si_config["n_labels"])

In [18]:
print("model:{}, audio_length:{}s, n_labels:{}".format(model, si_config['input_length']//16000, si_config["n_labels"]))

model:SimpleCNN, audio_length:1s, n_labels:62


In [31]:
os.makedirs("models/reddots/fewshot")

In [19]:
# si_model.load("models/reddots/si_reddots_frames_res8.pt")
si_config['n_epochs'] = 100
si_config['output_file'] = "../models/reddots/fewshot/si_reddots_frames_fewshot.pt"
train.train(si_config, model=si_model)

train step #99 accuracy: 0.4375, loss: 2.1804468631744385
epoch #0, final dev accuracy: 0.5077052345537758
saving best model...
train step #199 accuracy: 0.484375, loss: 1.517486572265625
train step #299 accuracy: 0.65625, loss: 1.124812364578247
epoch #1, final dev accuracy: 0.6130816170861937
saving best model...
train step #399 accuracy: 0.578125, loss: 1.2333470582962036
train step #499 accuracy: 0.828125, loss: 0.8116158246994019
epoch #2, final dev accuracy: 0.7160028127383676
saving best model...
train step #599 accuracy: 0.796875, loss: 0.665187418460846
train step #699 accuracy: 0.71875, loss: 0.8083511590957642
epoch #3, final dev accuracy: 0.7645344679633866
saving best model...
train step #799 accuracy: 0.828125, loss: 0.5745956301689148
train step #899 accuracy: 0.796875, loss: 0.5898747444152832
epoch #4, final dev accuracy: 0.829537328375286
saving best model...
train step #999 accuracy: 0.84375, loss: 0.46613532304763794
epoch #5, final dev accuracy: 0.8304252479023646


epoch #48, final dev accuracy: 0.9502407513348589
train step #8999 accuracy: 0.96875, loss: 0.07365724444389343
train step #9099 accuracy: 0.96875, loss: 0.12173281610012054
epoch #49, final dev accuracy: 0.9514743039664377
train step #9199 accuracy: 0.953125, loss: 0.09218963980674744
train step #9299 accuracy: 0.984375, loss: 0.05846339464187622
epoch #50, final dev accuracy: 0.9593702326468344
train step #9399 accuracy: 1.0, loss: 0.04073411226272583
train step #9499 accuracy: 1.0, loss: 0.026075541973114014
epoch #51, final dev accuracy: 0.9683268974065599
saving best model...
train step #9599 accuracy: 0.984375, loss: 0.07098622620105743
train step #9699 accuracy: 0.96875, loss: 0.09344206750392914
epoch #52, final dev accuracy: 0.9648884439359268
train step #9799 accuracy: 0.96875, loss: 0.0686812698841095
epoch #53, final dev accuracy: 0.9299020308924485
train step #9899 accuracy: 0.96875, loss: 0.08488106727600098
train step #9999 accuracy: 0.984375, loss: 0.052019014954566956


train step #17999 accuracy: 0.9375, loss: 0.0877956748008728
train step #18099 accuracy: 1.0, loss: 0.013708144426345825
epoch #98, final dev accuracy: 0.978898503051106
train step #18199 accuracy: 0.96875, loss: 0.10469090938568115
train step #18299 accuracy: 1.0, loss: 0.016465872526168823
epoch #99, final dev accuracy: 0.983016304347826
saving best model...
final test accuracy: 0.0


## SV Test

In [ ]:
enroll_spks = test_spks 